In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/code/phasor_julia`


In [2]:
include("module.jl")
using .PhasorNetworks

┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_potential :: Tuple{Real, AbstractVector, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.potential_to_phase :: Tuple{AbstractArray, Real, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_train :: Tuple{AbstractArray, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_potential :: Tuple{Real, AbstractVector, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243


┌ Warning: Replacing docs for `Main.PhasorNetworks.potential_to_phase :: Tuple{AbstractArray, Real, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_train :: Tuple{AbstractArray, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_potential :: Tuple{Real, AbstractVector, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.potential_to_phase :: Tuple{AbstractArray, Real, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_train :: Tuple{AbstractArray, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243


In [46]:
using LinearAlgebra: triu, diagm

In [66]:
using Statistics: std

In [67]:
function chance_level(nd::Int, samples::Int)
    symbol_0 = random_symbols((1, nd))
    symbols = random_symbols((samples, nd))
    sim = similarity_outer(symbol_0, symbols, 1) |> vec
    dev = std(sim)

    return dev
end

chance_level (generic function with 1 method)

In [68]:
chance_level(1024, 200)

0.02015479191116637

In [24]:
function generate_er_graph(n::Int, p::Real, self_loops::Bool = true)
    adj = rand(Float64, (n, n)) .< p
    #make undirected
    adj = triu(adj)
    adj =  (adj .+ adj') .> 0

    if !self_loops
        #remove self-loops
        for i in 1:n
            adj[i,i] = 0
        end
    end

    return adj
end

generate_er_graph (generic function with 2 methods)

In [25]:
g1 = generate_er_graph(20, 0.1)

20×20 BitMatrix:
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  1  0  0
 0  0  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  1  0  1  1  1  0  0  1  1  0  0  1  0  0  0
 0  0  0  0  1  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  1  0  0
 0  1  0  1  1  0  0  0  0  0  0  0  0  0  0  1  0  1  0  0
 0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
 0  1  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  1  1  0  0  0  0  0  0  1  0  0  0  0  0
 0  0  0  0  1  0  0  0

In [26]:
edges = findall(g1)

49-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 1)
 CartesianIndex(8, 2)
 CartesianIndex(10, 2)
 CartesianIndex(10, 3)
 CartesianIndex(13, 3)
 CartesianIndex(18, 3)
 CartesianIndex(5, 4)
 CartesianIndex(8, 4)
 CartesianIndex(4, 5)
 CartesianIndex(6, 5)
 ⋮
 CartesianIndex(16, 15)
 CartesianIndex(7, 16)
 CartesianIndex(8, 16)
 CartesianIndex(15, 16)
 CartesianIndex(5, 17)
 CartesianIndex(3, 18)
 CartesianIndex(7, 18)
 CartesianIndex(8, 18)
 CartesianIndex(11, 18)

In [27]:
e1 = edges[1]

CartesianIndex(1, 1)

In [40]:
function graph_to_vector(graph::AbstractMatrix, nd::Int)
    @assert size(graph,1) == size(graph,2) "Takes an adjacency matrix as the input"
    n = size(graph,1)
    #create vectors representing the nodes
    node_values = random_symbols((n, nd))
    #get cartesian coordinates representing each edge
    edges = findall(graph)
    n_edges = length(edges)
    edge_values = zeros(Float64, n_edges, nd)

    #iterate through the edges
    for (i, edge) in enumerate(edges)
        tx = edge[1]
        rx = edge[2]

        tx_symbol = node_values[tx,:]
        rx_symbol = node_values[rx,:]
        #create a representation for that edge by binding its incident nodes
        edge_symbol = PhasorNetworks.bind(tx_symbol, rx_symbol)
        edge_values[i,:] = edge_symbol
    end

    #combine the edges in the graph to the single embedding via bundling
    graph_embedding = bundle(edge_values, dims=1)
    return node_values, graph_embedding
end

graph_to_vector (generic function with 1 method)

In [42]:
nv1, gv1 = graph_to_vector(g1, 1024)

([-0.09215080738067627 -0.3930014371871948 … 0.2976473569869995 0.7505167722702026; -0.8252460956573486 -0.46050405502319336 … 0.28560328483581543 -0.5822343826293945; … ; 0.9176154136657715 0.3450801372528076 … 0.957565188407898 -0.21860146522521973; -0.7117501497268677 0.931100606918335 … 0.30991852283477783 0.9927903413772583], [-0.724975672529535 0.7470912680474266 … -0.4191979076492462 0.12671034108080745])

In [43]:
nv1 |> size

(20, 1024)

In [44]:
gv1

1×1024 Matrix{Float64}:
 -0.724976  0.747091  0.487704  0.308178  …  -0.19254  -0.419198  0.12671

In [ ]:
function query_edges(graph::AbstractMatrix, nodes::AbstractMatrix, threshold::Real = 3.0)
    n = size(nodes,1)

    for node in eachslice(nodes, dims=1)
        